# Pandas и большие файлы

In [1]:
import pandas as pd

In [ ]:
Домашнее задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv.
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок.
Под временем жизни понимается разница между максимальным
и минимальным значением столбца timestamp для данного значения userId.

In [81]:
df = pd.read_csv('Python_13_join/ml-latest-small/ratings.csv')

users = df.groupby('userId').count().sort_values('movieId', ascending = False)
active_users = users[(users['movieId']>= 100)]

min_max = df.groupby('userId').agg([min, max])
min_max['lifetime'] = min_max['timestamp', 'max'] - min_max['timestamp', 'min']
min_max.drop(['movieId', 'rating','timestamp'], axis='columns', inplace=True)
rez = active_users.join(min_max, how = 'left', on=['userId'])

rez[('lifetime', '')].mean() / 24 / 3600








455.2285713719898

In [ ]:
Домашнее задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

revenue_type = (rzd
    .merge(auto, how = 'outer', on='client_id' )
    .merge(air, how = 'outer', on='client_id')
    )
revenue_type.fillna(0)

revenue_type_address = (rzd
    .merge(auto, how = 'outer', on='client_id' )
    .merge(air, how = 'outer', on='client_id')
    .merge(client_base, how = 'outer', on='client_id')                    
    )
revenue_type_address.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


In [ ]:
Задание 3**
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей.
Т. е. для каждого user_id известна последовательность координат (широта/долгота), 
когда они требовались приложению для полноценной работы.
Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
1)- У каждого пользователя известен набор координат.
А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
ОТВЕТ: цифры, составляющие широту и долготу можно как-то обработать(сложить или перемножить),
чтобы получилось уникальное число, которое может служить кратким идентификатором пользователя. Зная порядок преобразований,
всегда можно обратно получить координаты пользователя, если надо.
2)- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга.
Как это отразится на вопросе расчетах пункта 1?
ОТВЕТ: Каждые отдельные координаты идентифицируют скорее визит или покупку, совершенные пользователем,
а не пользователя в целом. Поэтому на алгоритм обработки координат из вопроса 1 не отразится никак.
3)- Какие дополнительные признаки можно получить из координат?
Ведь это просто числа, которые сами по себе мало что дают.
ОТВЕТ: Координаты это точки на карте. Нанеся их на карту, 
    можно сортировать отдельных пользователей, выбранных по какому-то критерию по активности их перемещений в пространстве,
    можно сортировать пространства по популярности среди пользователей, всех или определенных
    можно анализировать как активность пользователя в пространстве влияет на его конверсию визитов в покупки
    и т.д., как одно влияет на другое и наоборот.

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [24]:
df = pd.read_csv('sales_db.csv')
df

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3
5,2,Авто,35
6,3,Дети,8
7,9,Дети,17


In [4]:
df.groupby('user_id').max()[['cost']]

,cost
user_id,
1,199
2,35
3,8
9,17


In [36]:
previous_user = None
user_data = []

with open('sales_db.csv') as f:
    f.readline()
    # previous_user = f.readline().strip(',')[0]
    for i, line in enumerate(f):
        user_id, category, cost = line.strip().split(',')
        cost = int(cost)
        print(user_id, category, cost)
        
        if previous_user:
            if previous_user == user_id:
                pass
            else:
#                 print(user_data)
#                 print(previous_user, max([x[2] for x in user_data]))
                print(previous_user, sorted(user_data, key=lambda x: -x[2])[0][1])
                print('\n')
                
                user_data = []
                previous_user = user_id
        else:
            previous_user = user_id
        
        user_data.append([user_id, category, cost])
            
print(user_data)
print(previous_user, max([x[2] for x in user_data]))
print(previous_user, sorted(user_data, key=lambda x: -x[2])[0][1])

1 Электроника 87
1 Дача 10
1 Дети 2
1 Дача 199
2 Авто 3
1 Дача


2 Авто 35
3 Дети 8
2 Авто


9 Дети 17
3 Дети


[['9', 'Дети', 17]]
9 17
9 Дети


# Объединение датафреймов

### Данные со слайдов

In [9]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [10]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [11]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [ ]:
visits.groupby('user_id').count().reset_index()

In [12]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [ ]:
purchases_pivot.reset_index()

In [13]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [14]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


In [ ]:
visits_grouped.join(purchases_pivot, how='left', on=['user_id'])

In [ ]:
visits_grouped.join(purchases_pivot, how='left')

In [ ]:
visits_grouped.merge(purchases_pivot, how='left', left_on=['user_id', 'source'], 
                     right_on=['user_name', 'category'])

### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [15]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [16]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [17]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [18]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [31]:
(clients
 .merge(earnings, how='left', on='id')  # объединение с доходами
 .merge(spending, how='left', on='id')
)

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
1,48329,Зарицкая Елизавета,12400,2500
2,51043,Благова Дарья,34500,22990
3,74943,Слепова Елена,89044,69880
4,75029,Гордецкий Максим,5355,6000


In [32]:
joined = clients.merge(earnings, how='left', on='id')
result = joined.merge(spending, how='left', on='id')

result[result.debit < result.credit]

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


In [30]:
result = clients.merge(earnings, how='left') 
result = result.merge(spending, how='left') 

result[result.credit > result.debit]

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [37]:
visits_grouped.join(purchases_pivot, how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [38]:
visits_grouped.join(purchases_pivot, how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


# Конкатенация таблиц

In [39]:
'123' + '456'

'123456'

In [40]:
[1, 2, 3] + [4, 5, 6]

[1, 2, 3, 4, 5, 6]

In [41]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [42]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [43]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [44]:
pd.concat([a, b])

/Users/kbashevoy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


In [47]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

,date,value_a,date,value_b
0,2020-01-01,1,2020-01-01,3
1,2020-01-02,2,2020-01-02,4
2,2020-01-03,3,2020-01-03,5


In [48]:
import os

In [ ]:
df = pd.DataFrame()

for file in os.listdir('folder_with_data'):
    # df_temp = pd.read_excel(file)
    df_temp = pd.read_csv(file)
    df.concat([df, df_temp])

### Дубликаты при объединении таблиц

In [49]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [50]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [51]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [52]:
movies.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)
movies.head()

,movieId,title,genres
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [53]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [54]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [55]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [60]:
visits.groupby('date').sum().reset_index().merge(orders, on='date', how='left')

,date,visits,orders
0,2019-11-01,18777,198
1,2019-11-02,22266,225
2,2019-11-03,19509,201


In [66]:
temp = (visits
 .pivot_table(index='date', columns='source', values='visits', aggfunc='sum')
 .reset_index()
 .fillna(0)
 .merge(orders, on='date', how='left')
)
temp

,date,organic,paid,orders
0,2019-11-01,16825.0,1952.0,198
1,2019-11-02,21890.0,376.0,225
2,2019-11-03,19509.0,0.0,201


In [67]:
temp['visits'] = temp['organic'] + temp['paid']
temp

,date,organic,paid,orders,visits
0,2019-11-01,16825.0,1952.0,198,18777.0
1,2019-11-02,21890.0,376.0,225,22266.0
2,2019-11-03,19509.0,0.0,201,19509.0


In [65]:
temp = visits.merge(orders, on='date', how='left')
temp

,date,source,visits,orders
0,2019-11-01,organic,16825,198
1,2019-11-01,paid,1952,198
2,2019-11-02,organic,21890,225
3,2019-11-02,paid,376,225
4,2019-11-03,organic,19509,201


In [ ]:
temp['']

In [56]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


### Оптимизация хранения данных

In [68]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [70]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [71]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
movieId    9125 non-null int64
title      9125 non-null object
genres     9125 non-null object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB


In [ ]:
Nan

In [69]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [72]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
len(ratings)

In [73]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
len(movies)

In [ ]:
len(ratings) == len(joined)

In [74]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [75]:
import numpy as np

In [76]:
genres = ['Drama', 'Action', 'Thriller']

In [77]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [78]:
len(ratings)

100004

In [79]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [80]:
len(movies)

9125

In [81]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [82]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [83]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

In [84]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [85]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

NameError: name 'genres_ratings' is not defined

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base